In [1]:
%matmodlab

Loading BokehJS ...

Populating the interactive namespace from matmodlab and bokeh


# Permutation

In this example, the bulk and shear moduli input parameters to the elastic material model are "permutated" to determine the material model sensitivity to their values.

In [2]:
def func(x, xnames, d, job, *args):
    """The objective function to exercise"""
    mps = MaterialPointSimulator(job)
    
    # set up the material
    parameters = dict(zip(xnames, x))
    mps.Material('elastic', parameters)

    mps.StrainStep(components=(1, 0, 0), increment=1., scale=-.05, frames=10)
    mps.StrainStep(components=(2, 0, 0), increment=1., scale=-.05, frames=10)
    mps.StrainStep(components=(1, 0, 0), increment=1., scale=-.05, frames=10)
    mps.StrainStep(components=(0, 0, 0), increment=1., scale=-.05, frames=10)


    s = mps.get('S.XX', 'S.YY', 'S.ZZ')
    p = -np.sum(s, axis=1) / 3.
    return np.amax(p)

In [5]:
N = 10
K = PermutateVariable('K', 125e9, method=PERCENTAGE, b=20, N=N)
G = PermutateVariable('G', 45e9, method=WEIBULL, b=14, N=N)
xinit = [K, G]
permutator = Permutator('permutate', func, xinit, method=ZIP,
                        descriptors=['MAX_PRES']), correlations=True)
permutator.run()

Running job 10 
Done


In [6]:
p = permutator.plot_correlations()
show(p)

IndexError: list index out of range